In [1]:
%%html
<style>
.h1_cell, .just_text {
    box-sizing: border-box;
    padding-top:5px;
    padding-bottom:5px;
    font-family: "Times New Roman", Georgia, Serif;
    font-size: 125%;
    line-height: 22px; /* 5px +12px + 5px */
    text-indent: 25px;
    background-color: #fbfbea;
    padding: 10px;
}

hr { 
    display: block;
    margin-top: 0.5em;
    margin-bottom: 0.5em;
    margin-left: auto;
    margin-right: auto;
    border-style: inset;
    border-width: 2px;
}
</style>

<h1>
<center>
Module 6: Random Forests
</center>
</h1>
<div class=h1_cell>

You will be working with the loan table again.

</div>

In [2]:
import pandas as pd
import os

week = 5

home_path =  os.path.expanduser('~')

file_path = '...'

file_name = 'loan_wrangled_w'+str(week)+'.csv'

loan_table = pd.read_csv(home_path + file_path + file_name)

In [3]:
os.chdir(home_path + '...')
!git pull

Already up-to-date.


In [4]:
#load the lbirary from content this week

import sys
sys.path.append(home_path + '...')
from week6 import *
%who function

accuracy	 build_pred	 build_tree_iter	 caser	 compute_prediction	 compute_training	 f1	 find_best_splitter	 forest_builder	 
generate_table	 gig	 gini	 informedness	 k_fold	 predictor_case	 probabilities	 seeder	 tree_predictor	 
vote_taker	 


In [5]:
pd.set_option('display.max_columns', None)
loan_table.head()

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status,no_lam,filled_lam,pa_Rural,pa_Semiurban,pa_Urban,pa_nan,lam_bin,lam_Low,lam_Average,lam_High,ch_bad,ch_good,ch_nan,apin_binned,apin_low,apin_average,apin_high,apin_nan,dep_0,dep_1,dep_2,dep_3+
0,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,1,1,146.412162,0,0,1,0,Low,1,0,0,0,1,0,low,1,0,0,0,1,0,0,0
1,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,0,0,128.000000,1,0,0,0,Low,1,0,0,0,1,0,low,1,0,0,0,0,1,0,0
2,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,1,0,66.000000,0,0,1,0,Low,1,0,0,0,1,0,low,1,0,0,0,1,0,0,0
3,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,1,0,120.000000,0,0,1,0,Low,1,0,0,0,1,0,low,1,0,0,0,1,0,0,0
4,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,1,0,141.000000,0,0,1,0,Low,1,0,0,0,1,0,low,1,0,0,0,1,0,0,0


In [6]:
loan_table.columns.values

array(['Gender', 'Married', 'Dependents', 'Education', 'Self_Employed',
       'ApplicantIncome', 'CoapplicantIncome', 'LoanAmount',
       'Loan_Amount_Term', 'Credit_History', 'Property_Area',
       'Loan_Status', 'no_lam', 'filled_lam', 'pa_Rural', 'pa_Semiurban',
       'pa_Urban', 'pa_nan', 'lam_bin', 'lam_Low', 'lam_Average',
       'lam_High', 'ch_bad', 'ch_good', 'ch_nan', 'apin_binned',
       'apin_low', 'apin_average', 'apin_high', 'apin_nan', 'dep_0',
       'dep_1', 'dep_2', 'dep_3+'], dtype=object)

<hr>
<h1>
1: Explore forest options
</h1>
<p>
<div class=h1_cell>
<p>
Check out the results you get from forests of size 5, 11, 17. Use default depth of 2.
<p>
First, define the columns to use. I do that for you below.
</div>

In [7]:
splitter_columns = [
        #Dependents
        'dep_0', 'dep_1', 'dep_2', 'dep_3+',
        #ApplicantIncome
       'apin_low', 'apin_high', 'apin_average',
        #Property_Area
        'pa_Rural', 'pa_Semiurban','pa_Urban',
        #LoanAmount
        'lam_Low', 'lam_Average', 'lam_High',
        #Credit_History
        'ch_bad', 'ch_good']

<hr>
<h2>
You might get different results
</h2>
<p>
<div class=h1_cell>
<p>
Since we are now in random land, your results may not match mine.
</div>

In [8]:

len(forest1)

5

In [9]:

print((accuracy(forest1_types), f1(forest1_types), informedness(forest1_types)))

(0.8094462540716613, 0.8764519535374868, 0.4104956556082149)


In [10]:

len(forest2)

11

In [11]:

print((accuracy(forest2_types), f1(forest2_types), informedness(forest2_types)))

(0.8094462540716613, 0.8764519535374868, 0.4104956556082149)


In [12]:

len(forest3)

17

In [13]:

print((accuracy(forest3_types), f1(forest3_types), informedness(forest3_types)))

(0.762214983713355, 0.8513238289205702, 0.25093799368088465)


<hr>
<h1>
2: Implement Out of Bag testing
</h1>
<p>
<div class=h1_cell>
<p>
Last module we looked at the use of K-Folding as a means to test our trees. Random Forests give us an alternative by using out of bag testing. Using notes from the content notebook this week, find a way to do prediction using the oob idea. As reminder, the set union of all the oob lists in a forest make up the testing set. If there is a row in loan_table that is not in any oob list, that row should be omitted from the test set. I am going to leave it to you to come up with an algorithm for doing oob testing. If you get totally stuck, I can supply hints.
<p>
Your goal is to be able to execute this code as is (e.g., for forest1):
<p>
<pre>
<code>
print((accuracy(forest1_oob_types), f1(forest1_oob_types), informedness(forest1_oob_types)))
</code>
</pre>
<p>
How you get to this point is up to you.
</div>

In [14]:
#Define new functions here


In [31]:

print(forest1_oob_types.sum())  # length of testing table
print((accuracy(forest1_oob_types), f1(forest1_oob_types), informedness(forest1_oob_types)))

550
(0.8054545454545454, 0.8742655699177438, 0.39691768180140263)


<hr>
<h1>Write it out</h1>
<div class=h1_cell>

Save the table so can use it in next module.
</div>

In [ ]:
import os

week = 6  # change this each week

home_path =  os.path.expanduser('~')

file_path = '...'

file_name = 'loan_wrangled_w'+str(week)+'.csv'

loan_table.to_csv(home_path + file_path + file_name, index=False)